In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP /Final project/ChineseHumorSentiment-master/data'
# GOOGLE_DRIVE_PATH = os.path.join('drive', 'Shared drives', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
# print(os.listdir(GOOGLE_DRIVE_PATH))
# sys.path.append(GOOGLE_DRIVE_PATH)

In [3]:
import pandas as pd

In [18]:
train_set = pd.read_csv("drive/Shared drives/NLP Final Project/Dataset/balanced_train_task1.csv",encoding='utf8')
test_set = pd.read_csv("drive/Shared drives/NLP Final Project/Dataset/balanced_test_task1.csv",encoding='utf8')
train_set = train_set.drop(columns=['id', 'Unnamed: 0'])


test_set = test_set.drop(columns=['id', 'Unnamed: 0'])

In [19]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Preprocessing

In [6]:
! pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 42.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 35.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
train_set

,joke,label
0,当他们全都被困在一起时，他们为什么称他们为“公寓”？,0
1,总有一个白痴比你指望的还要多。,0
2,我们的父母和祖父母是一个更大的威胁！,0
3,通过用细条培根覆盖任何切口和放牧，可以对猪进行轻微的皮肤移植。,0
4,有时我们不会想你。和它一起生活。,0
...,...,...
7995,四个食物组：快速，冷冻，速溶和巧克力。,0
7996,不要把我与事实混淆，我的思绪已经弥补了！,1
7997,财务科长对一雇员说：“你太太请求我们把你的月薪支票直接寄给她。” “为什么？” “她说取消中...,1
7998,如果橘子闻起来像鸡肉，为什么西红柿会变蓝？想一想！,0


In [20]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

example_text = train_set.iloc[400, 0]
bert_input = tokenizer(example_text,padding='max_length', max_length = 40, 
                       truncation=True, return_tensors="pt")

In [9]:
print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[ 101, 2111, 2094, 4638, 4495, 3833, 2218, 1008,  671, 2476, 5291, 8024,
         3680,  702,  782, 6963, 4522,  678,  749, 1313, 6381,  511,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [10]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] 孩 子 的 生 活 就 像 一 张 纸 ， 每 个 人 都 留 下 了 印 记 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [21]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
labels = {'true': 1,
          'false': 0
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['label']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 40, truncation=True,
                                return_tensors="pt") for text in df['joke']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [28]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.dropout = nn.Dropout(0.5)
        self.rnn = nn.LSTM(768, 200, num_layers = 3, device='cuda', batch_first=False, bidirectional=True, dropout=0.5)
        self.linear = nn.Linear(400, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        rnn_output, _ = self.rnn(dropout_output.view(len(dropout_output), 1, -1))
        dropout_output = self.dropout(rnn_output)
        linear_output = self.linear(dropout_output)
        dropout_output = self.dropout(linear_output)
        final_layer = (dropout_output).reshape((2, 2))
        return final_layer

In [29]:
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import f1_score
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate, weight_decay=1e-5)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f} \
                | Val Macro F1: {f1_score(output.argmax(dim=1).cpu().detach().numpy(), val_label.cpu().detach().numpy(), average="macro")}')
                  
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, train_set, test_set, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4000/4000 [09:46<00:00,  6.82it/s]


Epochs: 1 | Train Loss:  0.346                 | Train Accuracy:  0.517                 | Val Loss:  0.343                 | Val Accuracy:  0.574                 | Val Macro F1: 1.0


100%|██████████| 4000/4000 [09:44<00:00,  6.85it/s]


Epochs: 2 | Train Loss:  0.338                 | Train Accuracy:  0.614                 | Val Loss:  0.331                 | Val Accuracy:  0.683                 | Val Macro F1: 0.3333333333333333


100%|██████████| 4000/4000 [09:49<00:00,  6.79it/s]


...

Epochs: 5 | Train Loss:  0.292                 | Train Accuracy:  0.721                 | Val Loss:  0.291                 | Val Accuracy:  0.719                 | Val Macro F1: 1.0


100%|██████████| 4000/4000 [09:43<00:00,  6.85it/s]


Epochs: 6 | Train Loss:  0.278                 | Train Accuracy:  0.732                 | Val Loss:  0.278                 | Val Accuracy:  0.723                 | Val Macro F1: 1.0


  9%|▉         | 358/4000 [00:52<08:51,  6.86it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model,"drive/Shared drives/NLP Final Project/Lawrence model/lstm+bert")